### --------ASM591/ILS595 Geospatial Programming and Data Science--------------- 
# Week 6 Lab Exercises : Fill the missing temperature data in WHIN weather dataset using two interpolation aglorithms
In this lab, you will continue working on the WHIN station dataset (https://whin.org/about/). The WHIN stations capture daily weather data in ten counties (including Tippecanoe county) in North-Central Indiana. The daily weather records usually have some missing data from certain stations. In this lab exercise, you will apply two interpolation algorithms to fill those missing data in the database. 


In [1]:
# Materials created by Gang Shao and appropriately edited by Ben Hancock for lab exercises in ILS595/ASM591 Geospatial Programming 
# and Data Science, Spring 2021, contact gshao@purdue.edu for questions and bug reports

## Section 0: Setup

In [2]:
import os as os    # os library contains functionality related to operating system
path = os.getcwd() # os library was used to assign the current directory's file path to a variable
print(path)        # printing variable path

D:\GIS\Weather_Analysis_Case_Study


In [3]:
# The two data files we need in this lab are:
#   WHIN_current_02232021.csv - the stations data that contains weather information on Feb. 23, 2021
#   WHIN_stations.csv - the data that contains locations of weather stations

os.listdir()  # os library was used to list the contents of the current directory

['.ipynb_checkpoints', 'ImportLog', 'Index', 'Weather_Analysis_Case_Study.aprx', 'Weather_Analysis_Case_Study.gdb', 'Weather_Analysis_Case_Study.tbx', 'Week6Lab_edited_revB.ipynb', 'WHIN_current_02232021.csv', 'WHIN_stations.csv']

In [4]:
import arcpy         # this library provides access to the geoprocessing functionality of ArcGIS Pro
import pandas as pd  # Pandas is used to read tabular data into a data structure called a DataFrame
import numpy as np   # numpy is a numerical library used for creating arrays and perform mathematical calculations
import matplotlib.pyplot as plt  # matplotlib is a library for rndering a variety of statistical graphs

## Section 1: Data Wrangling
In this section, we will clean the weather tables for the interpolation analysis. Topics covered will include:
* Creating Pandas DataFrames using the two CSV datasets
* Exploring the datasets using methods available to DataFrames
* Identifying a unique common key between the datasets
* Performing a merge operation to combine the datasets into a single DataFrame
* Finding and correcting missing data where appropriate

In [5]:
# read the weather data on Feb. 23, 2021 in the file "WHIN_current_02232021.csv"
weather_data_df = pd.read_csv("WHIN_current_02232021.csv")
weather_data_df.head(5)

,station_id,latitude,longitude,name,observation_time,temperature,temperature_high,temperature_low,humidity,solar_radiation,solar_radiation_high,rain,rain_inches_last_hour,wind_speed_mph,wind_direction_degrees,wind_gust_speed_mph,wind_gust_direction_degrees,pressure,soil_temp_1,soil_temp_2,soil_temp_3,soil_temp_4,soil_moist_1,soil_moist_2,soil_moist_3,soil_moist_4
0,1,40.93894,-86.47418,WHIN001-PULA001,2021-02-23T23:45:00Z,35.0,37.0,35.0,86.0,1.0,7,0,0,5.0,225.0,6,225.0,29.990,32.0,32.0,32.0,34.0,4.0,7.0,6.0,15.0
1,3,40.73083,-86.98467,WHIN003-WHIT001,2021-02-23T23:45:00Z,35.0,35.0,35.0,85.0,1.0,5,0,0,3.0,180.0,5,202.5,29.932,32.0,32.0,33.0,33.0,7.0,8.0,6.0,3.0
2,4,40.04188,-86.76929,WHIN004-MONT001,2021-02-23T23:45:00Z,42.0,42.0,40.0,80.0,3.0,9,0,0,2.0,225.0,4,225.0,29.986,32.0,32.0,32.0,32.0,121.0,137.0,6.0,7.0
3,5,40.89261,-86.23791,WHIN005-CASS001,2021-02-23T23:30:00Z,39.0,39.0,37.0,80.0,11.0,14,0,0,8.0,225.0,12,225.0,29.961,32.0,33.0,34.0,34.0,6.0,4.0,5.0,5.0
4,6,41.03067,-86.75275,WHIN006-PULA002,2021-02-23T23:45:00Z,37.0,38.0,37.0,83.0,1.0,5,0,0,4.0,202.5,7,202.5,29.977,33.0,33.0,34.0,35.0,5.0,2.0,3.0,1.0


In [6]:
weather_data_df.shape

(136, 26)

In [7]:
# read the weather station data that we created in Week5's lab exercise
station_locations_df = pd.read_csv("WHIN_stations.csv")
station_locations_df.head(5)

,id,name,latitude,longitude
0,142,WHIN052-MONT004,40.10483,-86.86619
1,150,WHIN058-PULA007,41.00467,-86.68428
2,161,WHIN071-PULA009,40.97021,-86.89882
3,143,WHIN053-PULA005,40.98224,-86.38542
4,151,WHIN059-CASS006,40.84436,-86.18173


In [8]:
# by checking the data shape, we found there are 167 stations and we only have 136 stations with weather data in 'weather_data_df'
station_locations_df.shape

(167, 4)

By checking the data shape, we found there are 167 stations in 'station_locations_df' and we only have 136 stations with weather data in 'weather_data_df'.

### Data Exploration
Next, you will explore the datasets to determine if they have any unique common keys. Identifying a common key will allow you to merge the two datasets into a single DataFrame. The first step to finding a common key will be to check the DataFrames for unique values.

In [9]:
# you can use .columns to remind ourselves what columns are available in a DataFrame
station_locations_df.columns

Index(['id', 'name', 'latitude', 'longitude'], dtype='object')

In [10]:
# check how many unique values are in each field
# here you will loop over each column in the dataframe 
# and print out the column name followed by the number 
# of unique values
for i in station_locations_df.columns:
    print(f'{i}: {len(station_locations_df[i].unique())}')  # looks like id or lat/long are unique

id: 167
name: 130
latitude: 167
longitude: 167


In [11]:
# check how many unique values are in each field
for i in weather_data_df.columns:
    print(f'{i}: {len(weather_data_df[i].unique())}') # What does the f'...' do? (Hint: https://docs.python.org/3/tutorial/inputoutput.html)
    
# an individual column can also be checked for unique values
print('\n')  # print a newline
print(f'unique values in "name": {len(weather_data_df["name"].unique())}') # 107 unique names in feb 23 data vs 130 in whole dataset

station_id: 136
latitude: 136
longitude: 136
name: 107
observation_time: 3
temperature: 17
temperature_high: 17
temperature_low: 16
humidity: 31
solar_radiation: 10
solar_radiation_high: 10
rain: 1
rain_inches_last_hour: 1
wind_speed_mph: 10
wind_direction_degrees: 11
wind_gust_speed_mph: 12
wind_gust_direction_degrees: 11
pressure: 120
soil_temp_1: 9
soil_temp_2: 8
soil_temp_3: 8
soil_temp_4: 9
soil_moist_1: 52
soil_moist_2: 47
soil_moist_3: 38
soil_moist_4: 38


unique values in "name": 107


In [12]:
# "name" looked like a promising common key because it is present
# in both DataFrames, but you can tell that there are duplicate names
# because the number of unique names is fewer than the total number of
# records in the DataFrames

# for example, station_locations has 167 records, but only 130 unique names

# the station_id and id columns look promising, but you need to confirm they
# correspond to the same stations in each dataset first

# you will arbitrarily test id value "3"
print(weather_data_df[weather_data_df["station_id"] == 3])   # where station_id is 3 for Feb 23
print(station_locations_df[station_locations_df["id"] == 3]) # where id is 3 for station locations

# they have the same lat/long, so they are the same site

   station_id  latitude  longitude  ... soil_moist_2 soil_moist_3  soil_moist_4
1           3  40.73083  -86.98467  ...          8.0          6.0           3.0

[1 rows x 26 columns]
     id             name  latitude  longitude
134   3  WHIN003-WHIT001  40.73083  -86.98467


In [13]:
# you will now test the station_id and id "5"
idx = 5

# note: loc gets named locations, iloc get integer locations

# get locations in weather_data_df (Feb 23) where station_id is idx (5)
# and only get the station_id, name, and lat/long values
print(weather_data_df.loc[weather_data_df["station_id"] == idx, ["station_id", "name", "latitude", "longitude"]])

# show where whole dataset station is idx (5)
print(station_locations_df[station_locations_df["id"] == idx])

# they are the same, indicating you can use "station_id" in weather_data_df and "id" in station_locations_df
# you can use these two columns to merge the two DataFrames into a single DataFrame

   station_id             name  latitude  longitude
3           5  WHIN005-CASS001  40.89261  -86.23791
     id             name  latitude  longitude
136   5  WHIN005-CASS001  40.89261  -86.23791


### Table Merge
Before you can further explore the data for missing values, you need to merge these two tables to find out which stations have weather data or not.

Merging data requires a "key" to join the two tables. Station names usually are the first choice as a "key" in most cases, but the last lab in week 5 indicated that many stations shared the same name, so you are required to find out another field to use as the key. In the previous section "station_id" and "id" were identified as "key" for the merge.

In [14]:
# before performing the merge, you need to make sure that the common key 
# has the same name in both DataFrames

# you will rename "station_id" to "id" in weather_data_df for feb 23
weather_data_df = weather_data_df.rename(columns={"station_id" : "id"}) 

In [15]:
weather_data_df.columns  # there is now an "id" column

Index(['id', 'latitude', 'longitude', 'name', 'observation_time',
       'temperature', 'temperature_high', 'temperature_low', 'humidity',
       'solar_radiation', 'solar_radiation_high', 'rain',
       'rain_inches_last_hour', 'wind_speed_mph', 'wind_direction_degrees',
       'wind_gust_speed_mph', 'wind_gust_direction_degrees', 'pressure',
       'soil_temp_1', 'soil_temp_2', 'soil_temp_3', 'soil_temp_4',
       'soil_moist_1', 'soil_moist_2', 'soil_moist_3', 'soil_moist_4'],
      dtype='object')

In [16]:
# merge the station_locations_df DataFrame into 
# the weather_data_df DataFrame using the "id"
# as the common key and the merge method
weather_data_df = station_locations_df.merge(weather_data_df, how="outer", on="id", suffixes=("", "_2") )

weather_data_df.head(5)  

,id,name,latitude,longitude,latitude_2,longitude_2,name_2,observation_time,temperature,temperature_high,temperature_low,humidity,solar_radiation,solar_radiation_high,rain,rain_inches_last_hour,wind_speed_mph,wind_direction_degrees,wind_gust_speed_mph,wind_gust_direction_degrees,pressure,soil_temp_1,soil_temp_2,soil_temp_3,soil_temp_4,soil_moist_1,soil_moist_2,soil_moist_3,soil_moist_4
0,142,WHIN052-MONT004,40.10483,-86.86619,40.10483,-86.86619,WHIN052-MONT004,2021-02-23T23:45:00Z,45.0,45.0,44.0,71.0,3.0,7.0,0.0,0.0,3.0,225.0,5.0,247.5,29.149,32.0,33.0,34.0,36.0,5.0,7.0,3.0,2.0
1,150,WHIN058-PULA007,41.00467,-86.68428,41.00467,-86.68428,WHIN058-PULA007,2021-02-23T23:45:00Z,37.0,38.0,37.0,82.0,3.0,7.0,0.0,0.0,5.0,225.0,8.0,225.0,29.241,32.0,33.0,33.0,33.0,4.0,6.0,7.0,7.0
2,161,WHIN071-PULA009,40.97021,-86.89882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,143,WHIN053-PULA005,40.98224,-86.38542,40.98224,-86.38542,WHIN053-PULA005,2021-02-23T23:45:00Z,37.0,37.0,37.0,82.0,2.0,7.0,0.0,0.0,8.0,202.5,10.0,202.5,29.179,32.0,33.0,34.0,35.0,5.0,5.0,14.0,8.0
4,151,WHIN059-CASS006,40.84436,-86.18173,40.84436,-86.18173,WHIN059-CASS006,2021-02-23T23:45:00Z,39.0,40.0,39.0,76.0,2.0,7.0,0.0,0.0,6.0,202.5,8.0,225.0,29.123,32.0,32.0,32.0,32.0,2.0,191.0,12.0,14.0


In [17]:
weather_data_df.shape  

# why are there 168 rows now when largest dataframe had 167?

# there was a station id in weather_data_df that did not
# have a matching id in station_locations_df

# the merged table includes the 167 rows for which there 
# were id matches between the two tables plus a row for the id
# without a match

# because there was no match in station_locations_df for this id, 
# its row has NaN values for the station_locations_df columns

# you will address this row in the next section of the notebook

(168, 29)

### Data cleaning after merge
Now you will explore the merged data for missing values that may negatively impact the upcoming geoprocesses. The main interest will be focused on the latitude and longitude values and the temperature values. You will create a point feature class (vector) by using the latitude and longitude values so it is important that they are present. It will be useful to know how many of the temperature values are missing in the current dataset as those will be used during interpolation. 

In [18]:
weather_data_df.head(168)

,id,name,latitude,longitude,latitude_2,longitude_2,name_2,observation_time,temperature,temperature_high,temperature_low,humidity,solar_radiation,solar_radiation_high,rain,rain_inches_last_hour,wind_speed_mph,wind_direction_degrees,wind_gust_speed_mph,wind_gust_direction_degrees,pressure,soil_temp_1,soil_temp_2,soil_temp_3,soil_temp_4,soil_moist_1,soil_moist_2,soil_moist_3,soil_moist_4
0,142,WHIN052-MONT004,40.10483,-86.86619,40.10483,-86.86619,WHIN052-MONT004,2021-02-23T23:45:00Z,45.0,45.0,44.0,71.0,3.0,7.0,0.0,0.0,3.0,225.0,5.0,247.5,29.149,32.0,33.0,34.0,36.0,5.0,7.0,3.0,2.0
1,150,WHIN058-PULA007,41.00467,-86.68428,41.00467,-86.68428,WHIN058-PULA007,2021-02-23T23:45:00Z,37.0,38.0,37.0,82.0,3.0,7.0,0.0,0.0,5.0,225.0,8.0,225.0,29.241,32.0,33.0,33.0,33.0,4.0,6.0,7.0,7.0
2,161,WHIN071-PULA009,40.97021,-86.89882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,143,WHIN053-PULA005,40.98224,-86.38542,40.98224,-86.38542,WHIN053-PULA005,2021-02-23T23:45:00Z,37.0,37.0,37.0,82.0,2.0,7.0,0.0,0.0,8.0,202.5,10.0,202.5,29.179,32.0,33.0,34.0,35.0,5.0,5.0,14.0,8.0
4,151,WHIN059-CASS006,40.84436,-86.18173,40.84436,-86.18173,WHIN059-CASS006,2021-02-23T23:45:00Z,39.0,40.0,39.0,76.0,2.0,7.0,0.0,0.0,6.0,202.5,8.0,225.0,29.123,32.0,32.0,32.0,32.0,2.0,191.0,12.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,38,WHIN038-WARR004,40.29858,-87.38647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,41,WHIN041-TIPP006 Cumberland Gardens,40.46325,-86.91867,40.46325,-86.91867,WHIN041-TIPP006 Cumberland Gardens,2021-02-23T23:45:00Z,45.0,46.0,45.0,69.0,2.0,5.0,0.0,0.0,0.0,NaN,0.0,NaN,29.243,33.0,34.0,35.0,35.0,9.0,6.0,7.0,9.0
165,42,WHIN042-CARR002,40.54233,-86.48150,40.54233,-86.48150,WHIN042-CARR002,2021-02-23T23:45:00Z,39.0,40.0,39.0,75.0,2.0,7.0,0.0,0.0,2.0,202.5,3.0,202.5,29.210,32.0,33.0,33.0,35.0,7.0,4.0,4.0,6.0
166,44,Pedestrian Bridge,40.41936,-86.89753,40.41936,-86.89753,Pedestrian Bridge,2021-02-23T23:45:00Z,46.0,47.0,46.0,68.0,3.0,7.0,0.0,0.0,3.0,180.0,7.0,157.5,29.978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# check the coordinate columns as you need those for feature conversion 
# are there any records with null lat/long?

# the isnull() method will return the rows containing null values 
# for the specified column and the sum method adds up the returned rows
print(sum(weather_data_df['latitude'].isnull()))   # 1 record
print(sum(weather_data_df['longitude'].isnull()))  # 1 record

1
1


In [20]:
# check the row with missing coordinates
weather_data_df[weather_data_df['latitude'].isnull()]

# this is the record that did not match between the two tables

,id,name,latitude,longitude,latitude_2,longitude_2,name_2,observation_time,temperature,temperature_high,temperature_low,humidity,solar_radiation,solar_radiation_high,rain,rain_inches_last_hour,wind_speed_mph,wind_direction_degrees,wind_gust_speed_mph,wind_gust_direction_degrees,pressure,soil_temp_1,soil_temp_2,soil_temp_3,soil_temp_4,soil_moist_1,soil_moist_2,soil_moist_3,soil_moist_4
167,168,NaN,NaN,NaN,40.48079,-87.20682,WHIN051E-BENT005,2021-02-23T23:30:00Z,45.0,45.0,44.0,72.0,9.0,12.0,0.0,0.0,1.0,247.5,2.0,247.5,29.805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# both datasets contained lat/long values for the stations
# you will check if the associated values from the merged 
# dataset have the missing lat/long values

# find where in df latitude is null, and view the associated values from the Feb 23 columns (those that were merged)
weather_data_df.loc[weather_data_df["latitude"].isnull(), ["name_2", "latitude_2", "longitude_2"]].values

array([['WHIN051E-BENT005', 40.48079, -87.20682]], dtype=object)

In [22]:
# replace the missing coordinates and name using the values from the weather table fields
missing_values = weather_data_df.loc[weather_data_df["latitude"].isnull(), ["name_2", "latitude_2", "longitude_2"]].values
weather_data_df.loc[weather_data_df["latitude"].isnull(), ["name", "latitude", "longitude"]] = missing_values

In [23]:
# print the filled data
weather_data_df[weather_data_df["id"] == 168]

,id,name,latitude,longitude,latitude_2,longitude_2,name_2,observation_time,temperature,temperature_high,temperature_low,humidity,solar_radiation,solar_radiation_high,rain,rain_inches_last_hour,wind_speed_mph,wind_direction_degrees,wind_gust_speed_mph,wind_gust_direction_degrees,pressure,soil_temp_1,soil_temp_2,soil_temp_3,soil_temp_4,soil_moist_1,soil_moist_2,soil_moist_3,soil_moist_4
167,168,WHIN051E-BENT005,40.48079,-87.20682,40.48079,-87.20682,WHIN051E-BENT005,2021-02-23T23:30:00Z,45.0,45.0,44.0,72.0,9.0,12.0,0.0,0.0,1.0,247.5,2.0,247.5,29.805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# check if there are any additional missing lat/long records
weather_data_df.loc[weather_data_df['latitude'].isnull(),]
weather_data_df.loc[weather_data_df['longitude'].isnull(),]

,id,name,latitude,longitude,latitude_2,longitude_2,name_2,observation_time,temperature,temperature_high,temperature_low,humidity,solar_radiation,solar_radiation_high,rain,rain_inches_last_hour,wind_speed_mph,wind_direction_degrees,wind_gust_speed_mph,wind_gust_direction_degrees,pressure,soil_temp_1,soil_temp_2,soil_temp_3,soil_temp_4,soil_moist_1,soil_moist_2,soil_moist_3,soil_moist_4


In [25]:
# check for missing temperature data
print(sum(weather_data_df['temperature'].isnull()))  # 33 rows do not have temp

# display rows with missing temperature data
weather_data_df[weather_data_df['temperature'].isnull()]

# you do not need to take any action at this point

# just be aware these stations will not contribute 
# to the interpolation in the upcoming section

# if too many stations had been missing temperature values
# then the interpolation may not have been possible due
# to a lack of data points, but with only 33 missing you can continue

33


,id,name,latitude,longitude,latitude_2,longitude_2,name_2,observation_time,temperature,temperature_high,temperature_low,humidity,solar_radiation,solar_radiation_high,rain,rain_inches_last_hour,wind_speed_mph,wind_direction_degrees,wind_gust_speed_mph,wind_gust_direction_degrees,pressure,soil_temp_1,soil_temp_2,soil_temp_3,soil_temp_4,soil_moist_1,soil_moist_2,soil_moist_3,soil_moist_4
2,161,WHIN071-PULA009,40.970210,-86.898820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,152,WHIN060-WARR005,40.308780,-87.299430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,165,WHIN074-BENT007,40.485270,-87.503750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,148,WHIN051-BENT005,40.475640,-87.206890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2,WHIN002-BENT001,40.475700,-87.206920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,37,WHIN037-TIPP004,40.468060,-86.989660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,138,Wolcott,40.790660,-86.985167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,139,Francesville,41.022050,-86.833810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,140,Atkinson,40.563483,-87.213401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,53,Francesville,40.941671,-86.838294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# save the merged DataFrame as csv file
weather_data_df.to_csv(fr"{path}\all_stations_weather_0223.csv", index=False)

In [27]:
print(fr"{path}")

D:\GIS\Weather_Analysis_Case_Study


## Section 2: Data analysis: interpolation

### Create feature class with csv table

In [28]:
os.listdir()

['.ipynb_checkpoints', 'all_stations_weather_0223.csv', 'ImportLog', 'Index', 'Weather_Analysis_Case_Study.aprx', 'Weather_Analysis_Case_Study.gdb', 'Weather_Analysis_Case_Study.tbx', 'Week6Lab_edited_revB.ipynb', 'WHIN_current_02232021.csv', 'WHIN_stations.csv']

In [29]:
# Set the arcpy workspace to the root path for the ArcGIS Pro project
arcpy.env.workspace = path

# Generate a list of the available file geodatabase workspaces
workspaces = arcpy.ListWorkspaces("*", "FileGDB")

# Print the name of each availble file geodatabase
for index, workspace in enumerate(workspaces):
    print(fr"index: {index}, workspace: {os.path.basename(workspace)}")

index: 0, workspace: Weather_Analysis_Case_Study.gdb


In [30]:
# At a minimum, you should see the name of the default file geodatabase created
# with your ArcGIS Pro project - it will have the same name as your project

# Update the arcpy workspace to point to this default file geodatabase
# If there is more than one file geodatabase in your project,
# you may need to change the index value from 0 to the index value
# associated with the default file geodatabase
arcpy.env.workspace = workspaces[0]

# Going forward, the output generated by arcpy geoprocessing tools will be
# automatically stored in the default file geodatabase

# Set environment settings - Learn more about using environment settings with arcpy from here:
# https://pro.arcgis.com/en/pro-app/latest/arcpy/geoprocessing_and_python/using-environment-settings.htm

**Open a map in your project if one is not already open. As you run the following geoprocessing tools, 
the results will automatically be added to the map.**

In [31]:
# Set the local variables
in_table = fr"{path}\all_stations_weather_0223.csv"
out_feature_class = "WHIN_weather_0223"
x_coords = "longitude"
y_coords = "latitude"
z_coords = "temperature"

# Make the XY event layer...
# Learn more about the parameters used by XYTableToPoint from here:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/xy-table-to-point.htm
arcpy.management.XYTableToPoint(in_table, out_feature_class,
                                x_coords, y_coords, z_coords,
                                arcpy.SpatialReference(4326,115700)) #115700 allows vertical reference 

# the feature will show on the map as a layer

<Result 'D:\\GIS\\Weather_Analysis_Case_Study\\Weather_Analysis_Case_Study.gdb\\WHIN_weather_0223'>

In [32]:
## check the feature class just created - Learn more about the arcpy ListDatasets function from here:
# https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/listdatasets.htm
datasets = arcpy.ListDatasets(feature_type='feature')
datasets = [''] + datasets if datasets is not None else []

for ds in datasets:
    # Learn more about the arcpy ListFeatureClasses function from here:
    # https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-functions/listfeatureclasses.htm
    for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
        path1 = os.path.join(arcpy.env.workspace, ds, fc)
        print(path1)

D:\GIS\Weather_Analysis_Case_Study\Weather_Analysis_Case_Study.gdb\WHIN_weather_0223


### IDW interpolation

In [33]:
# Use arcpy.ListRasters to check your workspace for raster datasets

# Currently, you do not have any raster datasets, so ListRasters
# returns a non-iterable None object

# Learn more about the arcpy ListRasters function from here: https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/listrasters.htm
rasters = arcpy.ListRasters("*")

for raster in rasters:
    print(raster)

In [34]:
# IDW interpolation

# Set local variables
inPointFeatures = "WHIN_weather_0223"
zField = "temperature"
outLayer = "IDW_layer"
cellSize = 500.0 #how was this chosen?
power = 2

# Set variables for search neighborhood
majSemiaxis = 3000
minSemiaxis = 3000
angle = 0
maxNeighbors = 15
minNeighbors = 5
sectorType = "ONE_SECTOR"

# Learn more about the arcpy SearchNeighborhoodStandard function from here:
# https://pro.arcgis.com/en/pro-app/latest/arcpy/classes/searchneighborhoodstandard-class-.htm
searchNeighbourhood = arcpy.SearchNeighborhoodStandard(majSemiaxis, minSemiaxis,
                                                       angle, maxNeighbors,
                                                       minNeighbors, sectorType)

# Execute IDW - Learn more about the local variable and search neighborhood parameters used by IDW from here:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/geostatistical-analyst/idw.htm
idw_result = arcpy.IDW_ga(inPointFeatures, zField, outLayer, cell_size=cellSize, 
             power=power, search_neighborhood=searchNeighbourhood)

In [35]:
# Currently the IDW result exists as a Geostatistical layer
# Learn more about Geostatistical layers here:
# https://pro.arcgis.com/en/pro-app/2.8/help/analysis/geostatistical-analyst/what-is-a-geostatistical-layer-.htm

# It will now be converted to a raster dataset and stored 
# inside the file geodatabase workspace
arcpy.GALayerToRasters_ga(idw_result, "IDW_raster")

id,value
0,D:\GIS\Weather_Analysis_Case_Study\Weather_Analysis_Case_Study.gdb\IDW_raster
1,


### Extract interpolation temperature for each station

In [36]:
# Checking the raster data created
rasters = arcpy.ListRasters("*")
for raster in rasters:
    print(raster) #Now there is IDW raster

IDW_raster


In [37]:
# Description: Extracts the cells of a raster based on a set of points.
# Requirements: Spatial Analyst Extension
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/extract-values-to-points.htm

# Import system modules
from arcpy.sa import *

# Set local variables
inPointFeatures = "WHIN_weather_0223"
inRaster = "IDW_raster"
outPointFeatures = "WHIN_weather_0223_IDW"

# Execute ExtractValuesToPoints
ExtractValuesToPoints(inPointFeatures, inRaster, outPointFeatures,
                      "INTERPOLATE", "VALUE_ONLY")

<geoprocessing server result object object at 0x000002A70FBF9F30>

In [38]:
# check feature class created - Learn more about the arcpy ListFeatureClasses function from here:
# https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/listfeatureclasses.htm
features = arcpy.ListFeatureClasses("*")
for fc in features:
    print(fc)

WHIN_weather_0223
WHIN_weather_0223_IDW


### Data cleaning: Rename the field of interpolated temperature

In [39]:
# print out the field names of created feature class - Learn more about the arcpy ListFields function from here:
# https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/listfields.htm
fields = arcpy.ListFields("WHIN_weather_0223_IDW")
for field in fields:
    print(field.name)

OBJECTID
Shape
id
name
latitude
longitude
latitude_2
longitude_2
name_2
observation_time
temperature
temperature_high
temperature_low
humidity
solar_radiation
solar_radiation_high
rain
rain_inches_last_hour
wind_speed_mph
wind_direction_degrees
wind_gust_speed_mph
wind_gust_direction_degrees
pressure
soil_temp_1
soil_temp_2
soil_temp_3
soil_temp_4
soil_moist_1
soil_moist_2
soil_moist_3
soil_moist_4
RASTERVALU


In [40]:
# change the current name of interpolated value 'RASTERVALU' to a meanful one
# 'RASTERVALU' is the interpolated temperature value
new_field_name="Temp_IDW"
new_alias_name = "Temp_IDW_alt"

print(fc)

fieldList = arcpy.ListFields("WHIN_weather_0223_IDW")  #get a list of fields for the feature class
for field in fieldList: #loop through each field
    if field.name.lower() == 'rastervalu':  #look for the name (after converting to lower case)
        # rename the field - learn more about the arcpy AlterField function from here: 
        # https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/alter-field-properties.htm
        arcpy.AlterField_management(fc, field.name, new_field_name, new_field_alias = new_alias_name) #fc is WHIN_weather_0223_IDW
        print('New field name: {0}'.format(new_field_name))

WHIN_weather_0223_IDW
New field name: Temp_IDW


In [41]:
# print out the field names again to double check
fields = arcpy.ListFields("WHIN_weather_0223_IDW")
for field in fields:
    print(field.name) #RASTERVALU is now Temp_IDW
    
    #However, when check the attribute table, still says RASTERVALU

OBJECTID
Shape
id
name
latitude
longitude
latitude_2
longitude_2
name_2
observation_time
temperature
temperature_high
temperature_low
humidity
solar_radiation
solar_radiation_high
rain
rain_inches_last_hour
wind_speed_mph
wind_direction_degrees
wind_gust_speed_mph
wind_gust_direction_degrees
pressure
soil_temp_1
soil_temp_2
soil_temp_3
soil_temp_4
soil_moist_1
soil_moist_2
soil_moist_3
soil_moist_4
Temp_IDW


## Exercise: Kriging interpolation (10 points)
Now you will work on kriging interpolation on your own based on the demo above and add the results as "Temp_Kriging" to the resulted feature class. The final feature class will include both "Temp_IDW" and "Temp_Kriging".

Use the follow link as reference to write the script.<br />
Name: EmpiricalBayesianKriging_Example_02.py<br />
https://pro.arcgis.com/en/pro-app/latest/tool-reference/geostatistical-analyst/empirical-bayesian-kriging.htm

## The below code sample was created by a student. They came to the following solution by reading the materials in the reference link provided above and referring back to the steps taken when invoking IDW using arcpy.

In [42]:
# Name: EmpiricalBayesianKriging_Example_02.py
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/geostatistical-analyst/empirical-bayesian-kriging.htm
# Description: Bayesian kriging approach whereby many models created around the
# semivariogram model estimated by the restricted maximum likelihood algorithm is used.
# Requirements: Geostatistical Analyst Extension
# Author: Esri
# Student: Rachel Cook

# Set local variables
inPointFeatures = "WHIN_weather_0223_IDW"
zField = "temperature"
outLayer = "kriging_layer"
cellSize = 1000.0 #how was this chosen?

# Set variables for search neighborhood
radius = 500
smooth = 0.6

# Create search neighborhood - Learn more about the arcpy SearchNeighborhoodSmoothCircular function from here:
# https://desktop.arcgis.com/en/arcmap/10.3/analyze/arcpy-classes/searchneighborhoodsmoothcircular-class-.htm
searchNeighbourhood = arcpy.SearchNeighborhoodSmoothCircular(radius, smooth)

# Execute kriging 
ebk_result = arcpy.EmpiricalBayesianKriging_ga(in_features = inPointFeatures, 
                                  z_field = zField, 
                                  out_ga_layer = outLayer,
                                  cell_size = cellSize)

In [43]:
# Convert the EBK geostatistical layer to a raster dataset
arcpy.GALayerToRasters_ga(ebk_result, "kriging_raster")

id,value
0,D:\GIS\Weather_Analysis_Case_Study\Weather_Analysis_Case_Study.gdb\kriging_raster
1,


In [44]:
# What raster layers exist now
rasters = arcpy.ListRasters("*")
for raster in rasters:
    print(raster) #Now there is IDW raster and kriging raster

IDW_raster
kriging_raster


In [45]:
# Description: Extracts the cells of a raster based on a set of points.
# Requirements: Spatial Analyst Extension
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/extract-values-to-points.htm

# Set local variables
inPointFeatures = "WHIN_weather_0223_IDW"
inRaster = "kriging_raster"
outPointFeatures = "WHIN_weather_0223_kriging"

# Execute ExtractValuesToPoints
ExtractValuesToPoints(inPointFeatures, inRaster, outPointFeatures,
                      "INTERPOLATE", "VALUE_ONLY")

<geoprocessing server result object object at 0x000002A70B064B40>

In [46]:
# print out the field names of created feature class
fields=arcpy.ListFields("WHIN_weather_0223_kriging")
for field in fields:
    print(field.name) #Must rename RASTERVALU

OBJECTID
Shape
id
name
latitude
longitude
latitude_2
longitude_2
name_2
observation_time
temperature
temperature_high
temperature_low
humidity
solar_radiation
solar_radiation_high
rain
rain_inches_last_hour
wind_speed_mph
wind_direction_degrees
wind_gust_speed_mph
wind_gust_direction_degrees
pressure
soil_temp_1
soil_temp_2
soil_temp_3
soil_temp_4
soil_moist_1
soil_moist_2
soil_moist_3
soil_moist_4
Temp_IDW
RASTERVALU


In [47]:
# change the current name of interpolated value 'RASTERVALU' to a meanful one
# 'RASTERVALU' is the interpolated temperature value
new_field_name="Temp_kriging"
new_alias_name = "Temp_kriging_alt"

#Name feature class we are changing a field name for
fc = "WHIN_weather_0223_kriging"

fieldList = arcpy.ListFields("WHIN_weather_0223_kriging")  #get a list of fields for the feature class
for field in fieldList: #loop through each field
    if field.name.lower() == 'rastervalu':  #look for the name (after converting to lower case)
        arcpy.AlterField_management(fc, field.name, new_field_name, new_field_alias = new_alias_name)
        print('New field name: {0}'.format(new_field_name))

New field name: Temp_kriging


In [48]:
# print out the field names again to make sure it worked
fields = arcpy.ListFields("WHIN_weather_0223_kriging")
for field in fields:
    print(field.name) #success!

OBJECTID
Shape
id
name
latitude
longitude
latitude_2
longitude_2
name_2
observation_time
temperature
temperature_high
temperature_low
humidity
solar_radiation
solar_radiation_high
rain
rain_inches_last_hour
wind_speed_mph
wind_direction_degrees
wind_gust_speed_mph
wind_gust_direction_degrees
pressure
soil_temp_1
soil_temp_2
soil_temp_3
soil_temp_4
soil_moist_1
soil_moist_2
soil_moist_3
soil_moist_4
Temp_IDW
Temp_kriging


# Plese submit the whole project folder in a zip file, including the created feature class with "Temp_IDW" and "Temp_Kriging" and this notebook with your kriging script.

## Section 3: Data Visualization

### 3.1 Visualize temperature distribution

 <3.1>. Plot histograms of observed temperature and two interpolated temperatures in one figure. 
(Hints: stacked histograms with transparency or individual histograms in subplots) 

In [49]:
# Reference code, 
# the file "WHIN_weather_0223_IDW_EBK_Counties.csv" contains original observation temperature and both interpolated temperatures using IDW and EBK
# read the weather data as pandas dataframe
dfm = pd.read_csv('WHIN_weather_0223_IDW_EBK_Counties.csv')
dfm.head()

# convert temperature columns to numeric data type and force None values to become NaNs
dfm['Temp_IDWalt'] = pd.to_numeric(dfm['Temp_IDWalt'], errors='coerce')
dfm['Temp_EBKalt'] = pd.to_numeric(dfm['Temp_EBKalt'], errors='coerce')
dfm['temperature'] = pd.to_numeric(dfm['temperature'], errors='coerce')

In [50]:
#plot histogram 
#set variables
temp_observed = dfm['temperature']
temp_kriging = dfm['Temp_EBKalt']
temp_IDW = dfm['Temp_IDWalt']

fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(temp_observed, bins=np.arange(34, 50, 0.5), alpha = 0.2, color= 'yellow')
ax.hist(temp_kriging, bins=np.arange(34, 50, 0.5), alpha = 0.2, color= 'green')
ax.hist(temp_IDW, bins=np.arange(34, 50, 0.5), alpha = 0.4, color= 'pink')
ax.legend(('temp_observed', 'temp_kriging', 'temp_IDW'), loc='upper right')
plt.title('Observed Tempreture VS Interpolation')
plt.xlabel('Temperature')


plt.show()

In [51]:
#import libraries for data visualization
import seaborn as sns
import matplotlib.pyplot as plt

[![postimgcc: Figure 3.1](https://i.postimg.cc/bvm0V6WY/Figure-3-1.png)](https://postimg.cc/GBssHjb6)

### 3.2 Investigate relationships between observed temperature and two interpolated  temperatures

<br><3.2.1>. data visuals of your choice to present the relationships <br/> 
<br><3.2.2>. discuss which interpolation performs better and why (statistical analysis is encouraged but not required) <br/>
<br><3.2.3>. discuss potential ways to improve the interpolation results.   (hint: this is an open discussion) <br/>

In [52]:
# Reference code
#Observed vs Kriging
plt.scatter(temp_observed, temp_kriging,  alpha = 0.4, color = ['green'] )
plt.title('Observed vs Kriging')
plt.xlabel('Observed Temperature')
plt.ylabel('Kriging')

plt.show()

#Observed vs IDW
plt.scatter(temp_observed, temp_IDW, alpha = 0.5, color = ['pink'] )
plt.title('Observed vs IDW')
plt.xlabel('Observed Temperature')
plt.ylabel('IDW')
plt.show()

#3.2.2 #according to the plots below we can tell IDW interpolation is better (better fit with the observed data)

[![postimgcc: Figure 3.2a](https://i.postimg.cc/XJZ5Hn9x/Figure-3-2a.png)](https://postimg.cc/qhTggVbC)

[![postimgcc: Figure 3.2b](https://i.postimg.cc/d3ryFnHy/Figure-3-2b.png)](https://postimg.cc/wRxvc5dq)

### 3.3 Data visuals at county level

<br><3.3.1>. create temperature boxplots for each county (hint: consider using seaborn boxplot for better presentation) <br/>
<br><3.3.2>. find the ‘hottest’ and ‘coolest’ counties <br/> 
<br><3.3.3>. find the county with most varied temperatures. <br/>

In [53]:
#Reference code
plt.figure(figsize=(8,10))
plt.subplot(3,1,1)
sns.boxplot(x="NAME_L", y="temperature", data=dfm, palette="Set1")
plt.xticks([])
plt.xlabel('')
plt.title('Observed Temperature')
plt.subplot(3,1,2)
sns.boxplot(x="NAME_L", y="Temp_IDWalt", data=dfm, palette="Set1")
plt.xticks([])
plt.xlabel('')
plt.title('IDW Interpolation')
plt.subplot(3,1,3)
sns.boxplot(x="NAME_L", y="Temp_EBKalt", data=dfm, palette="Set1")
plt.title('EBK Interpolation')
plt.xticks(rotation=45)
plt.show()

[![postimgcc: Figure 3.3](https://i.postimg.cc/ZqnvnJsZ/Figure-3-3.png)](https://postimg.cc/DWVwY3KY)